In [1]:
import pandas as pd
import string
from nltk.corpus import stopwords

In [6]:
#read data from csv file 
df = pd.read_csv("imdb_labelled.txt",sep='\t',names=['comment','label'])
df


,comment,label
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1
...,...,...
743,I just got bored watching Jessice Lange take h...,0
744,"Unfortunately, any virtue in this film's produ...",0
745,"In a word, it is embarrassing.",0
746,Exceptionally bad!,0


In [8]:
df.head(10)

,comment,label
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1
5,"The rest of the movie lacks art, charm, meanin...",0
6,Wasted two hours.,0
7,Saw the movie today and thought it was a good ...,1
8,A bit predictable.,0
9,Loved the casting of Jimmy Buffet as the scien...,1


In [9]:
df.describe()


,label
count,748.000000
mean,0.516043
std,0.500077
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   comment  748 non-null    object
 1   label    748 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 11.8+ KB


In [11]:
#view data statistics using describe()
df.groupby('label').describe()

comment                                
        count unique                 top freq
label                                        
0         362    361  Not recommended.      2
1         386    384             10/10      2

In [12]:
#view columns of the dataset
df.columns

Index(['comment', 'label'], dtype='object')

In [15]:
#verify length of message and add it as a new column

df['length'] = df['comment'].apply(len)

df.head()

,comment,label,length
0,"A very, very, very slow-moving, aimless movie ...",0,87
1,Not sure who was more lost - the flat characte...,0,99
2,Attempting artiness with black & white and cle...,0,188
3,Very little music or anything to speak of.,0,44
4,The best scene in the movie was when Gerardo i...,1,108


In [17]:
#start text processor with vectorizer

from sklearn.feature_extraction.text import CountVectorizer

In [18]:
#define a class to get rid of stopwords

def nowords(comt):
    message = [char for char in comt if char not in string.punctuation]
    message = ''.join(message)
    return [words for words in message if words.lower() not in stopwords.words('english')]

In [19]:
#verifing function
df['comment'].head().apply(nowords)

0    [ , v, e, r,  , v, e, r,  , v, e, r,  , l, w, ...
1    [N,  , u, r, e,  , w, h,  , w,  , r, e,  , l, ...
2    [e, p, n, g,  , r, n, e,  , w, h,  , b, l, c, ...
3    [V, e, r,  , l, l, e,  , u, c,  , r,  , n, h, ...
4    [h, e,  , b, e,  , c, e, n, e,  , n,  , h, e, ...
Name: comment, dtype: object

In [23]:
#bag of words by applying the function and fit the data into it

bag_of_words = CountVectorizer(analyzer = nowords).fit(df['comment'])

In [24]:
#apply transform method to bag of words

bag_of_words_transformer = bag_of_words.transform(df['comment'])

In [ ]:
#apply tfidf transformer and fit the bag of words into (transformed)

from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer().fit(bag_of_words)

In [26]:
#view shape
b = tfidf_transformer.transform(bag_of_words_transformer)

b.shape

(748, 52)

In [39]:
#choose naive bayes model to detect the spam and fit the data into it

from sklearn.naive_bayes import MultinomialNB

spam_model  =  MultinomialNB().fit(b,df['label'])

In [40]:
#check model for the predicted and expected value say for comment #1 and comment #5

#for comment 1

comment = df['comment'][1]

bag_of_words_for_comment  = bag_of_words.transform([comment])

tfidf =  tfidf_transformer.transform(bag_of_words_for_comment)

print("predicted",spam_model.predict(tfidf)[0])
print("expected",df.label[1])

predicted 0
expected 0


In [41]:
#for comment 5

comment  = df['comment'][4]

bag_of_words_for_comment = bag_of_words.transform([comment])

tfidf = tfidf_transformer.transform(bag_of_words_for_comment)

print("predicted",spam_model.predict(tfidf)[0])
print("expected",df.label[4])

predicted 1
expected 1
